In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch

C:\Users\mydma\miniconda3\envs\myenv2\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\mydma\miniconda3\envs\myenv2\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


C:\Users\mydma\miniconda3\envs\myenv2\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer.pad_token = tokenizer.eos_token 

In [3]:
# Define your dataset class
class SimpleDataset(Dataset):
    def __init__(self, input_texts, target_texts, tokenizer, max_length):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Tokenize input and target text with specified max length
        input_encodings = self.tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        target_encodings = self.tokenizer(target_text, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)


        # Remove unnecessary keys from target_encodings
        target_encodings["input_ids"] = target_encodings.pop("input_ids")

        return {
            "input_ids": input_encodings["input_ids"].squeeze(),
            "attention_mask": input_encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
        }


In [4]:
# Your input and target texts
input_texts = [
    "Who is your boss?", 
    "Hello",
] * 50  # Repeat for more instances - increase this number if needed

target_texts = [
    "You are my boss.",
    "What is your problem, human?",
] * 50  # Repeat for more instances - increase this number if needed

In [5]:
# Set a maximum sequence length for both input and output
max_length = 20

# Create the dataset
dataset = SimpleDataset(input_texts, target_texts, tokenizer, max_length)


In [8]:
import time
# Define training arguments
output_dir_cp = f'./flan_t5_fine_tuned_10e-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir=output_dir_cp,
    num_train_epochs=10,  # Adjust as needed 
    per_device_train_batch_size=1,  # Adjust as needed
    auto_find_batch_size=True,
    learning_rate=1e-5,
    # weight_decay=0.01,
    logging_steps=500,
    save_steps=500,
)

# Initialize the Trainer with default collate_fn
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [7]:
# Fine-tune the model
trainer.train()
output_dir="./flan_t5_fine_tuned_10e_local"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Step,Training Loss
500,0.251400
1000,0.004200


('./flan_t5_fine_tuned_10e_local\\tokenizer_config.json',
 './flan_t5_fine_tuned_10e_local\\special_tokens_map.json',
 './flan_t5_fine_tuned_10e_local\\spiece.model',
 './flan_t5_fine_tuned_10e_local\\added_tokens.json',
 './flan_t5_fine_tuned_10e_local\\tokenizer.json')

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))